<a href="https://colab.research.google.com/github/tutur90/RAG/blob/main/Copie_de_Project_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project : RAG

The goal of this project is to create a simple LLMs based RAG module. Obviously the most complex part is how to correctly link all components. We let you free to create your own database. Anything can be ragged, from research paper, songs, subtitles, books,...

This project is voluntarly sparsely helped, as, as engineers, you will dig into lots of existing method, and will need to pick the best one up. We want you to get familiar with the engineering world.

We will give you some recommendations. You will see lots of issues during this project, some you've already seen during Labs, other that are new. So buckle up, read the docs, and RAG your data.

Obviously there are lots of tutorials of the internet that you could just copy paste to get a baseline.


## **We encourage you to do code versioning using Github.**


**Ideal Project Timeline:**

*   Talking and getting to know the Modules. Discussing about the choice of your LLMs and the environment you'll have. Choosing a first set of data to RAG. Setting up your Github (Optional) (1h)
*   Setting up your first RAG Chain using Langchain or other (2-3h)
*   Understand the limitation of your RAG and find enhancements to set up your 2nd RAG. (2h)
*   Unveiling the unknown document, adapt your RAGs (2h)
*   Deploy it (Optional)
*   Begin your presentation (2h)
*   Presentation (5 min/groups)


We'll evaluate your presentation quality, your RAG system's capability, and your progress throughout the project sessions.

**Students who missed the first session will start with a score of 0 in the progress part**. :-)


Presentation:
- The number of slides you can do is unlimited
- You only have 5 min to present your project. We will stop you at 5 min whether you've finished or not
- Your presentation should include:
  - a presentation of your workflow (Agile Methodology, What's the job of each one...)
  - a presentation of your final pipeline with all enhancements done
  - a proof a work of your RAG on your data, and on the unknown data
  - what limitations you have and how to tackle them. For each too obvious limitations (more GPUs, more RAM..) : -1
  - if you've deployed your RAG, a scannable QR code to live test it.

#  Preliminaries : Some useful downloads

We give you some useful frameworks, that you could use to build your RAG.

In [ ]:
!pip install -q pypdf python-dotenv
!pip install transformers
!pip install -q datasets loralib sentencepiece
!pip install -q einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama-index
!%pip install --upgrade --quiet  langchain langchain-openai faiss-cpu tiktoken
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain.chains import RetrievalQA

# I - Data Sourcing

Data Sourcing for RAG is really simple. Some questions to guide you:

* What data do we need ?
* How can we correctly parse the data ?
* Does the vector index provides a good representation for the vector database ?
* For example, using FAISS, can you easily retrieve your document ?

To begin, pick a simple story and store it in a Vector Database. You have the choice between multiple VectorStores (ChromaDB, QDrant,...)


In [ ]:
%mkdir data

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader


loader = PyPDFDirectoryLoader("data")
data = loader.load()
len(data)


1

In [ ]:
# Create an instance of the RecursiveCharacterTextSplitter class with specific parameters.
# It splits text into chunks of 1000 characters each with a 150-character overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

# 'data' holds the text you want to split, split the text into documents using the text splitter.
docs = text_splitter.split_documents(data)

# II - Module Creation

Should you create a RAG Module, you need or not a LLM. We encourage you to test some LLMs (Mistral, LLama, Falcon, Gemma, ...) However, be aware that you won't have the space to run it on this colab.

We highly recommend to use LangChain, to build your Q&A app.

Some Questions to guide you:
* What model is easily accesible
* Are there any existing code to begin with ?
* What about the prompts ?
* What about the document parsing ?


In [ ]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
modelPath='BAAI/bge-large-zh-v1.5'

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
db = FAISS.from_documents(docs, embeddings)

In [ ]:
question = "Did Denka breach the ERAs? "
searchDocs = db.similarity_search(question)
print(searchDocs[0].page_content)

இதயம்  என்பது  இஷ ்காவில்  உள்ள  அம்பு  
इश्क़ में दिल फ़ना है, oh-oh-oh 
நீக்கவும்  அல்லது  உருவாக்கவும்  
मैंने तुझको  चुना है, oh-oh-oh 
يرتدي كل ألوانك، ويرتدي أسلوبك  
तेरा हुआ मैं सब को छोड़ के, oh-oh-oh 
"కొలవడం  ద్వా రా , రహస్యా లను  బహిరగతం చేయడం  ద్వా రా  ప్రేమలో  పడకండి ." 
आया हूँ मैं सब को बोल के, oh-oh-oh 
Ôi em đã đến bên anh, s ức lực của anh đã cạn kiệt 
ਤੇਰਾ ਹੋਇਆ  ਮੈਂ, ਯਾਰ ਵੇ, ਭ ੁੱਲਿਆ  ਏ ਸੰਸਾਰ  ਵੇ 
Το κουπί  σου έφυγε , η δύναμή σου έφυγε  
Έγινα  δικός  σου, φίλε μου, ξέχασα  αυτόν  τον κόσμο 
Έφυγα  από τον κόσμο για σένα, ένωσα  την καρδιά  μου μαζί σου 
अब तेरा मैं तो हो गया, पाक े तुझे मैं खो गया 
നിനക്കായി  ഞാൻ  ഈ ല ാകം  വിട്ടുല ായി , നിലനാട ാപ്പം  എടെ  ഹൃദയം  
ലേർത്തു . 
ഇലപ്പാൾ  ഞാൻ  നിങ്ങളുല താണ്  (ഞാൻ   ൂർത്തിയാക്കി ), നിങ്ങടള  
കടെത്തിയതിന്  ലേഷം  ഞാൻ  നഷ്ടടപ്പട്ടു , നഷ്ടടപ്പട്ടു , അടത  
इश्क़ में दिल बना है 
इश्क़ में दिल फ़ना है, oh-oh-oh 
हूँसा िे या रुला िे 
मैंने तुझको  चुना है, oh-oh-oh 
िुदनया  कहती , "इश्क़ भूल है, बे-दफ़जूल  है"


## Models

In [ ]:
from torch import bfloat16
import transformers

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    torch_dtype=bfloat16,
    device_map='auto',

)
model.eval()

/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1341: UserWarning: Current model requires 520097664 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-29-39a5ca355ec1>", line 21, in <cell line: 21>
    uvicorn.run(app, port=8000)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 575, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
pip = pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=False,  # if using langchain set True
    task="text-generation",
    # we pass model parameters here too
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1,  # if output begins repeating increase
    do_sample=True,
)
llm = HuggingFacePipeline(
    pipeline=pip,
    )

In [ ]:
question = "Why I need to love? "
llm.invoke(question)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'\n\nI\'m not sure if this is the right place for this question, but I\'ll ask it anyway. I\'ve been thinking about this a lot lately and I can\'t seem to find an answer that makes sense to me. I\'ve heard people say things like "love is what makes life worth living" or "you should love everyone because they are all part of God\'s creation", but those answers don\'t really satisfy me. I understand that love can bring happiness and joy into our lives, but why do we need that? Why can\'t we just be content with existing without loving anyone or anything? And why do we need to love everyone? Is it really necessary to love every single person in the world, even the ones who have done us harm? I feel like there must be some deeper reason for why we need to love, but I just can\'t figure it out. Any insights would be greatly appreciated.\nUser 1: Love is the force that binds us together as a species. It\'s what makes us social animals. We are wired to connect with each other, to care for eac

## Retrivers

In [ ]:
# Create a retriever object from the 'db' using the 'as_retriever' method.
# This retriever is likely used for retrieving data or documents from the database.
retriever = db.as_retriever()

docs = retriever.get_relevant_documents(question)
print(docs[0].page_content)

இதயம்  என்பது  இஷ ்காவில்  உள்ள  அம்பு  
इश्क़ में दिल फ़ना है, oh-oh-oh 
நீக்கவும்  அல்லது  உருவாக்கவும்  
मैंने तुझको  चुना है, oh-oh-oh 
يرتدي كل ألوانك، ويرتدي أسلوبك  
तेरा हुआ मैं सब को छोड़ के, oh-oh-oh 
"కొలవడం  ద్వా రా , రహస్యా లను  బహిరగతం చేయడం  ద్వా రా  ప్రేమలో  పడకండి ." 
आया हूँ मैं सब को बोल के, oh-oh-oh 
Ôi em đã đến bên anh, s ức lực của anh đã cạn kiệt 
ਤੇਰਾ ਹੋਇਆ  ਮੈਂ, ਯਾਰ ਵੇ, ਭ ੁੱਲਿਆ  ਏ ਸੰਸਾਰ  ਵੇ 
Το κουπί  σου έφυγε , η δύναμή σου έφυγε  
Έγινα  δικός  σου, φίλε μου, ξέχασα  αυτόν  τον κόσμο 
Έφυγα  από τον κόσμο για σένα, ένωσα  την καρδιά  μου μαζί σου 
अब तेरा मैं तो हो गया, पाक े तुझे मैं खो गया 
നിനക്കായി  ഞാൻ  ഈ ല ാകം  വിട്ടുല ായി , നിലനാട ാപ്പം  എടെ  ഹൃദയം  
ലേർത്തു . 
ഇലപ്പാൾ  ഞാൻ  നിങ്ങളുല താണ്  (ഞാൻ   ൂർത്തിയാക്കി ), നിങ്ങടള  
കടെത്തിയതിന്  ലേഷം  ഞാൻ  നഷ്ടടപ്പട്ടു , നഷ്ടടപ്പട്ടു , അടത  
इश्क़ में दिल बना है 
इश्क़ में दिल फ़ना है, oh-oh-oh 
हूँसा िे या रुला िे 
मैंने तुझको  चुना है, oh-oh-oh 
िुदनया  कहती , "इश्क़ भूल है, बे-दफ़जूल  है"


In [ ]:
# Create a retriever object from the 'db' with a search configuration where it retrieves up to 4 relevant splits/documents.
retriever = db.as_retriever(search_kwargs={"k": 1})

# Create a question-answering instance (qa) using the RetrievalQA class.
# It's configured with a language model (llm), a chain type "refine," the retriever we created, and an option to not return source documents.
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

In [ ]:
question="What's the topic of document"
result = qa.invoke({"query": question})
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'query': "What's the topic of document", 'result': 'The topic of the document appears to be a song or poem in multiple languages including Tamil, Hindi, Urdu, and Arabic. The lyrics seem to express feelings of love and longing towards someone, with references to losing oneself in that person and forgetting the world around them. There are also mentions of the heart and the passage of time. Overall, it seems like a romantic composition.'}


In [ ]:
print(result['result'])

The topic of the document appears to be a song or poem in multiple languages including Tamil, Hindi, Urdu, and Arabic. The lyrics seem to express feelings of love and longing towards someone, with references to losing oneself in that person and forgetting the world around them. There are also mentions of the heart and the passage of time. Overall, it seems like a romantic composition.


In [ ]:
question="Find the the original music from the translations of the document"
result = qa.invoke({"query": question})
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'query': 'Find the the original music from the translations of the document', 'result': "\nThe provided text appears to be lyrics for a song in multiple languages. However, without any additional context or information, it's impossible to determine the original music or source of these lyrics. The translations suggest that the song may have originated in India or South Asia, but this is just speculation based on the language usage. Without further information, such as the title of the song, the artist or composer, or any identifying metadata, it would be difficult to accurately identify the original music."}


In [ ]:
question="Give an exhastive list of languages of the document"
result = qa.invoke({"query": question})
print(result['result'])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'query': 'Give an exhastive list of languages of the document', 'result': "The document appears to be written in Hindi and Urdu languages with English transliteration provided for some words. The text is a poem that seems to express love and longing towards someone. The title of the poem is 'Ishq Mein Dil Fanaa' which translates to 'Love has consumed my heart'. The poetic expressions are used extensely throughout the poem, making it difficult to determine if there are any other languages present without further analysis or context. Therefore, based on the given context, the document can be said to contain Hindi, Urdu, and English languages."}


In [ ]:
question="Language"
result = qa.invoke({"query": question})
print(result['result'])

NameError: name 'qa' is not defined

# III - Model Serving (Optional and for the best)

You can inspire yourself from the first hands-on, if your feeling powerful, you can build something using fastapi and push your deployed RAG into a simple github.io instance. Or just use the gradio deploiement framework.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install "fastapi[all]"

from fastapi import FastAPI, Request
from datetime import datetime
import logging

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 20.0 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

In [ ]:
app = FastAPI()

logging.basicConfig(level=logging.INFO, filename='app.log',
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Define a middleware for logging requests
@app.middleware("http")
async def log_requests(request: Request, call_next):
    request_time = datetime.now()
    # Log the request details
    logging.info(f"Request start: {request.method} {request.url}")

    # Proceed with the request
    response = await call_next(request)

    # Log the response details along with the duration
    process_time = (datetime.now() - request_time).total_seconds()
    logging.info(f"Request completed: {request.method} {request.url} Status: {response.status_code} Completed in {process_time}s")

    return response


class Question(BaseModel):
    query: str

# Assuming initialization of 'llm' and 'db' as shown in the user's original code snippet
retriever = db.as_retriever(search_kwargs={"k": 1})
qa_system = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever, return_source_documents=False)

@app.post("/answer/")
async def answer_question(question: Question):
    result = qa_system.invoke({"query": question.query})
    if not result or 'result' not in result or not result['result']:
        return {"answer": "Unable to find an answer."}
    return {"answer": result['result']}

In [ ]:
!pip install pyngrok
!pip install requests

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
auth_token = "2e99E1Zo2uJk1QyUz0AA1mEwu3f_3JzXBYu8vWUjWc6vRzZaE"

# Set the authtoken
ngrok.set_auth_token(auth_token)

# Connect to ngrok
ngrok_tunnel = ngrok.connect(8000)

# Print the public URL
print('Public URL:', ngrok_tunnel.public_url)

# Apply nest_asyncio
nest_asyncio.apply()

# Run the uvicorn server
uvicorn.run(app, port=8000)

Exception in thread Thread-13 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 139, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 184: ordinal not in range(128)
INFO:     Started server process [804]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://eaca-34-141-168-153.ngrok-free.app


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [804]


KeyboardInterrupt: 